# Iteration 2: Introduce elective patients

## Plan English Summary

The model also includes an arrival and treatment process for elective patients.  In real life elective surgeries where a patient will require critical care post surgery are cancelled if there are no beds available.  In the model this is simulated in a simple way: elective patients are generated and as they arrive the model logic checks to see if any beds are available.  If no beds are available the elective patient leaves the model immediately - this is called "balking" in queuing theory. If beds are available the elective patient takes one and begins treatment.

This is the first test of more complicated logic for the LLM.




## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## Show query 

In [2]:
display(Markdown("inputs/iteration_2.md"))

### Main command

Modify the model to add in an elective admissions treatment process.

Show the full code listing for the model.  Do not exclude any functions or classes.

### general simulation logic

Add a new arrival source to the CCU: Elective surgery

Elective surgery patients are modelled as a separate process from the unplanned admissions, but share the critical care bed resources. 

As an elective patient arrives to the CCU a check is made on the number of critical care beds available.  There are two outcomes from this check

Outcome 1: the number of beds in use is equal to the total number of beds available. In this case the elective patient leaves the model immediately. This is called a "cancelled operation" event and should be reported to the user.

Outcome 2. the number of beds in use is less than the total number of beds available. In this case the elective patient requests a critical care bed, is treated, and is then discharged. Before the bed is released for another patient there is an additional delay for intensive cleaning of the area and the bed.

### simulation inputs

the interarrival time of elective surgery patients has a normal distribution with mean 17.91 and standard deviation 3.16.

the treatment time of elective surgery patients is exponential mean: 57.34

## Date and time of input to LLM

> 1645 28th November 2023 

## Observations on the LLM Output

* The code outputted excluded the patient generator functions from the previous version of the code.  An additional input was needed.

## LLM Output 1:

In [2]:
import simpy
import numpy as np

# Define the critical care unit (CCU) model with unplanned admissions and elective surgery
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department', 'Elective surgery']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0, 17.91]
        self.stay_distributions = [(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), (57.34)]
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=24)

    # Generator function for patient arrival from Elective surgery
    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(17.91, 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
            else:
                self.env.process(self.elective_surgery_process(self.stay_distributions[5]))

    # Elective surgery process
    def elective_surgery_process(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(57.34)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Unplanned admissions treatment process
    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.process(self.patient_arrival_elective_surgery())
        env.run(until=self.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)

# modified by tester as produces error.
# ccu_model.run()


## ADDITIONAL COMMAND/INPUT

the generator functions for unplanned arrival types are missing in the code above

### Observations on the LLM output

* The generator functions are again included.
* Two new functions have been added to the code `patient_arrival_elective_surgery` and `elective_surgery_process`
* `patient_arrival_elective_surgery` includes a line of code intended to handle the balking of elective surgery patients.  As requested balking happens before requesting a resource.

```python
if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
    print(...)
```
* elective intera-arrival times follows a normal distribution and treatment time follows an exponential as specified.
* The elective treatment time parameters have been added to the `stay_distributions` but it is also hard coded in the sampling.
* The code and model runs without a runtime error.


## Sources reported by the LLM

* https://towardsdatascience.com/simulation-with-simpy-287cbdf5d78e
* https://www.youtube.com/watch?v=TALKZZV0TiU
* https://towardsdatascience.com/object-oriented-discrete-event-simulation-with-simpy-53ad82f5f6e2
* https://github.com/capocchi/DEVSimPy
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7491890/


## LLM Output 2

This code adds an elective admissions treatment process to the critical care unit (CCU) model. Elective surgery patients are modeled as a separate process from the unplanned admissions but share the critical care bed resources. Upon arrival, a check is made on the number of critical care beds available, leading to either a "cancelled operation" event if no beds are available or the patient undergoing treatment and being discharged if beds are available. The model prints out useful information after each event.

The interarrival time of elective surgery patients follows a normal distribution with a mean of 17.91 and a standard deviation of 3.16. The treatment time of elective surgery patients follows an exponential distribution with a mean of 57.34.

The code creates a class `CCUModel` that defines the CCU model with the addition of elective surgery as a new arrival source. It includes a generator function for patient arrival from elective surgery, an elective surgery process, an unplanned admissions treatment process, and generator functions for patient arrival from other sources. The simulation environment is created using `simpy.Environment()`, and an instance of the `CCUModel` is created to run the simulation.

The code is based on the requirements specified and uses SimPy 4.1.1 and Python 3.10.4, as requested. For more details on SimPy and Python, please refer to the official documentation and release notes.

Citations:
[1] https://towardsdatascience.com/simulation-with-simpy-287cbdf5d78e

[2] https://youtube.com/watch?v=TALKZZV0TiU

[3] https://towardsdatascience.com/object-oriented-discrete-event-simulation-with-simpy-53ad82f5f6e2

[4] https://github.com/capocchi/DEVSimPy

[5] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7491890/

In [4]:
import simpy
import numpy as np

# Define the critical care unit (CCU) model with unplanned admissions and elective surgery
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department', 'Elective surgery']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0, 17.91]
        self.stay_distributions = [(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), (57.34)]
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=24)

    # Generator function for patient arrival from Elective surgery
    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(17.91, 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
            else:
                self.env.process(self.elective_surgery_process(self.stay_distributions[5]))

    # Elective surgery process
    def elective_surgery_process(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(57.34)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Generator function for patient arrival from Accident and Emergency
    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(22.72))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[0]))

    # Generator function for patient arrival from the Wards
    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(26.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[1]))

    # Generator function for patient arrival from Emergency surgery
    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(37.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[2]))

    # Generator function for patient arrival from other hospitals
    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(47.2))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[3]))

    # Generator function for patient arrival from the X-Ray department
    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(575.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[4]))

    # Unplanned admissions treatment process
    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.process(self.patient_arrival_elective_surgery())
        env.run(until=self.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)
ccu_model.run()


Patient 1 arrived from other hospitals at time 14.289754152855629
Patient 1 admitted to critical care bed at time 14.289754152855629
Elective surgery patient 2 arrived at time 15.376267460963936
Elective surgery patient 2 admitted to critical care bed at time 15.376267460963936
Patient 3 arrived from other hospitals at time 18.257143160759558
Patient 3 admitted to critical care bed at time 18.257143160759558
Patient 4 arrived from the Wards at time 18.35351186804083
Patient 4 admitted to critical care bed at time 18.35351186804083
Patient 5 arrived from Accident and Emergency at time 21.588160737879935
Patient 5 admitted to critical care bed at time 21.588160737879935
Patient 6 arrived from the Wards at time 24.32577835286834
Patient 6 admitted to critical care bed at time 24.32577835286834
Patient 7 arrived from Accident and Emergency at time 26.402153205338013
Patient 7 admitted to critical care bed at time 26.402153205338013
Patient 8 arrived from the Wards at time 30.96472664610380

## Modifications to LLM code to test for correct model outputs.

* **Extreme value test 1**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted and all electives are cancelled.
    * Actual result (PASS): after 24 arrivals queues form and no patients are admitted and all electives are cancelled.

* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result (PASS): The only type of patient to arrive to the model is "Elective."


### Extreme value test 1

PASS

In [5]:
M = 10_000_000

import simpy
import numpy as np

# Define the critical care unit (CCU) model with unplanned admissions and elective surgery
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department', 'Elective surgery']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0, 17.91]
        self.stay_distributions = [(M, 267.51), (M, 276.54), (M, 218.02), (M, 457.67), (M, 108.67), (57.34)]
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=24)

        # ADDED FOR TESTING
        self.admissions = 0

    # Generator function for patient arrival from Elective surgery
    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(17.91, 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
            else:
                self.env.process(self.elective_surgery_process(self.stay_distributions[5]))

    # Elective surgery process
    def elective_surgery_process(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            self.admissions += 1
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(M)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Generator function for patient arrival from Accident and Emergency
    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(22.72))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[0]))

    # Generator function for patient arrival from the Wards
    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(26.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[1]))

    # Generator function for patient arrival from Emergency surgery
    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(37.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[2]))

    # Generator function for patient arrival from other hospitals
    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(47.2))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[3]))

    # Generator function for patient arrival from the X-Ray department
    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(575.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[4]))

    # Unplanned admissions treatment process
    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            self.admissions += 1
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.process(self.patient_arrival_elective_surgery())
        env.run(until=self.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)
ccu_model.run()

print(f'TOTAL ADMISSIONS: {ccu_model.admissions}')


Patient 1 arrived from the Wards at time 3.7425235195887288
Patient 1 admitted to critical care bed at time 3.7425235195887288
Elective surgery patient 2 arrived at time 16.573005308220452
Elective surgery patient 2 admitted to critical care bed at time 16.573005308220452
Patient 3 arrived from Accident and Emergency at time 21.290181275487733
Patient 3 admitted to critical care bed at time 21.290181275487733
Patient 4 arrived from Accident and Emergency at time 27.807318746745622
Patient 4 admitted to critical care bed at time 27.807318746745622
Elective surgery patient 5 arrived at time 34.29718275383294
Elective surgery patient 5 admitted to critical care bed at time 34.29718275383294
Patient 6 arrived from the Wards at time 48.91816391506178
Patient 6 admitted to critical care bed at time 48.91816391506178
Patient 7 arrived from other hospitals at time 54.19273699169485
Patient 7 admitted to critical care bed at time 54.19273699169485
Elective surgery patient 8 arrived at time 55.0

### Extreme value test 2

PASS.

M = 10_000_000

In [6]:
import simpy
import numpy as np

# Define the critical care unit (CCU) model with unplanned admissions and elective surgery
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department', 'Elective surgery']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0, 17.91]
        self.stay_distributions = [(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), (57.34)]
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=24)

    # Generator function for patient arrival from Elective surgery
    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(17.91, 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
            else:
                self.env.process(self.elective_surgery_process(self.stay_distributions[5]))

    # Elective surgery process
    def elective_surgery_process(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(57.34)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Generator function for patient arrival from Accident and Emergency
    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[0]))

    # Generator function for patient arrival from the Wards
    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[1]))

    # Generator function for patient arrival from Emergency surgery
    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[2]))

    # Generator function for patient arrival from other hospitals
    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[3]))

    # Generator function for patient arrival from the X-Ray department
    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[4]))

    # Unplanned admissions treatment process
    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.process(self.patient_arrival_elective_surgery())
        env.run(until=self.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)
ccu_model.run()


Elective surgery patient 1 arrived at time 13.12468794142434
Elective surgery patient 1 admitted to critical care bed at time 13.12468794142434
Elective surgery patient 2 arrived at time 27.81240390689647
Elective surgery patient 2 admitted to critical care bed at time 27.81240390689647
Elective surgery patient 3 arrived at time 42.896561217666935
Elective surgery patient 3 admitted to critical care bed at time 42.896561217666935
Elective surgery patient 3 discharged from critical care bed at time 43.602887287672026
Critical care bed is available for next patient at time 48.602887287672026
Elective surgery patient 4 arrived at time 64.24764394413164
Elective surgery patient 4 admitted to critical care bed at time 64.24764394413164
Elective surgery patient 4 discharged from critical care bed at time 72.19770511657947
Elective surgery patient 4 discharged from critical care bed at time 74.87708509476944
Critical care bed is available for next patient at time 77.19770511657947
Critical ca